In [1]:
import warnings
warnings.filterwarnings("ignore")

import pickle
import os
from ctgan import load_demo
from ctgan import CTGANSynthesizer

MODELS_PATH = './models'

C:\Users\Eli\Anaconda3\envs\tf1\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:
data = load_demo()

discrete_columns = [
    'workclass',
    'education',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'native-country',
    'income'
]

In [3]:
ctgan = CTGANSynthesizer(batch_size=400)
ctgan.fit(data, discrete_columns, epochs=3)

Todo: something with confidence_level
Epoch 1, Loss G: 1.7758, Loss D: -0.2754
Epoch 2, Loss G: 1.2241, Loss D: -0.1004
Epoch 3, Loss G: 0.8650, Loss D: -0.0680


In [11]:
samples = ctgan.sample(1000)
samples.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,34,?,147057,12th,9,Married-civ-spouse,Adm-clerical,Own-child,White,Female,64,3,31,United-States,<=50K
1,47,?,220634,HS-grad,13,Married-civ-spouse,Exec-managerial,Own-child,White,Male,117,-2,32,United-States,<=50K
2,17,?,444396,10th,14,Never-married,Priv-house-serv,Not-in-family,White,Female,48,-2,64,United-States,<=50K
3,26,Private,190604,Bachelors,7,Married-civ-spouse,Protective-serv,Husband,Black,Male,75,-3,40,Mexico,<=50K
4,32,Private,248270,Some-college,9,Married-spouse-absent,Sales,Not-in-family,White,Female,128,-2,40,Poland,<=50K


# Train CTGAN with confidence level and black-box model

### CTGAN code structure:
1. **transformer.py** - implements _Mode-specific Normalization_ to continuous values, while categorical are encoded with OHE.

2. **synthesizer.py** - main code, implements fit (training GAN), predict (sample data), save and load. 

3. **models.py** - file contains Discriminator, Generator and Residual models.

4. **sampler.py** - TODO

5. **conditional.py** - TODO

In [12]:
# load rf model
with open(os.path.join(MODELS_PATH, 'adult_rf_0.852.pkl'), 'rb') as rf_fd:
    rf_model = pickle.load(rf_fd)

In [ ]:
rf_ctgan = CTGANSynthesizer(batch_size=500, 
                            blackbox_model=rf_model , # blackbox_model should implement fit, predict, predict_proba
                            )

rf_ctgan.fit(data, discrete_columns, 
             epochs=3, 
             confidence_level=0.7) 